In [2]:
import os
import sys
import json
import time
import requests

from dotenv import load_dotenv

def setup_env():
    
    env_path = os.path.join(os.getcwd(), '../.env')

    if os.path.exists(env_path):
        load_dotenv(dotenv_path=env_path)
        
        print(f"Loaded environment variables from: \033[94m{env_path}\033[0m")
    else:
            print("\033[91mError: .env file not found. Please create one with your OPENAI_API_KEY.\033[0m")
            sys.exit(1)


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader # 웹에서 데이터를 가져옵니다
from langchain_community.vectorstores import Chroma # 텍스트 데이터를 벡터화하고 검색 가능한 벡터 저장소를 구축합니다
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain import hub # 랭체인의 허브 기능으로 사전 정의된 체인, 도구, 프롬프트 등을 사용

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
llm = ChatOpenAI(model="gpt-4o", temperature=0, streaming=True)

embed_model = OpenAIEmbeddings()

/tmp/ipykernel_38635/4075182730.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o", temperature=0, streaming=True)
/tmp/ipykernel_38635/4075182730.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embed_model = OpenAIEmbeddings()


In [5]:
prompt = hub.pull("rlm/rag-prompt")

generation_chain = prompt | llm

In [6]:
urls = [
    "https://www.promptingguide.ai/kr",
    "https://www.promptingguide.ai/kr/introduction/basics",
] # 텍스트 데이터를 가져올 웹 페이지의 URL 목록

docs = [WebBaseLoader(url).load() for url in urls]

docs_list = [item for sublist in docs for item in sublist]

In [7]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=0
) # 긴 텍스트 데이터를 일정한 크기의 청크(chunks)로 분할, 각 청크의 최대 크기를 512 토큰으로 설정

doc_splits = text_splitter.split_documents(docs_list) # docs_list에 있는 텍스트 데이터를 chunk_size=512 크기로 분할

In [ ]:
vectorstore = Chroma.from_documents(documents=doc_splits,
                                    collection_name="chroma",
                                    persist_directory="/", # 해당 위치는 임시 저장소이므로 영구 저장을 위해서는 경로를 변경해야 합니다
                                    embedding=embed_model) # 텍스트 문서를 벡터화하고 저장소를 생성

PermissionError: [Errno 13] Permission denied: '/content'

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2}) # 벡터 저장소를 검색할 수 있는 검색기(retriever) 생성